In [1]:
import kagglehub

kagglehub.dataset_download('hlgsagar1234567/vr-go')
kagglehub.dataset_download("adityaav80/validatiioon")

'/kaggle/input/validatiioon'

In [2]:
import pandas as pd
df = pd.read_csv("/kaggle/input/validatiioon/val_red.csv")
print(len(df))
df.head(5)

50050


,image_id,image_path,question,answer
0,61ee2320.jpg,/kaggle/input/vr-go/go/new/images/small/61/61e...,What is the main color of the phone case?,Black
1,61ee2320.jpg,/kaggle/input/vr-go/go/new/images/small/61/61e...,What is the object depicted on the phone case?,UFO
2,61ee2320.jpg,/kaggle/input/vr-go/go/new/images/small/61/61e...,Where is the camera lens positioned on the pho...,Top
3,61ee2320.jpg,/kaggle/input/vr-go/go/new/images/small/61/61e...,How many stars are visible around the UFO on t...,Seven
4,61ee2320.jpg,/kaggle/input/vr-go/go/new/images/small/61/61e...,What is the phone case covering?,Phone


In [3]:
!pip install hf_xet bert_score

In [4]:
from transformers import AutoProcessor, BlipForQuestionAnswering
from peft import PeftModel
import torch

# Load processor
processor = AutoProcessor.from_pretrained(
    "Salesforce/blip-vqa-base",
    trust_remote_code=True,
    use_fast=True
)

# Load base model (PyTorch)
model = BlipForQuestionAnswering.from_pretrained(
    "Salesforce/blip-vqa-base",
    torch_dtype=torch.float16,
    device_map="auto"
)


# Load PEFT finetuned model weights from your checkpoint
model = PeftModel.from_pretrained(
    model,
    "adityaav80/blip-basevqa-finetuned",
    subfolder="epoch-7",
    is_trainable=False
)

2025-05-16 04:56:42.394647: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747371402.455381     146 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747371402.473378     146 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [5]:
device = "cuda" if torch.cuda.is_available() else "cpu"
new_model = model.to(device)

In [6]:
device

'cpu'

In [ ]:
import pandas as pd
import torch
from sentence_transformers import SentenceTransformer, util
from PIL import Image
import re
import os
import transformers
from bert_score import score
from transformers import logging

logging.set_verbosity_error()

def clean_text(text):
    return re.sub(r'[^a-zA-Z0-9\s]', '', str(text).lower().strip())

st,en = 1,2
start_row = int(50050/10)*st+1
end_row =  int(50050/10)*en
output_csv = f"/kaggle/working/evaluation_{st}_{en}.csv"

baai = SentenceTransformer("BAAI/bge-base-en-v1.5")

results = []
image_path = None
for i in range(start_row, min(end_row, len(df))):
    try:

        if i % 10 == 0 or image_path is None:
            image_path = df.loc[i, "image_path"]
            image = Image.open(image_path).convert("RGB")

        question = df.loc[i, "question"]
        gt = df.loc[i, "answer"]

        prompt = f"{question}"

        inputs = processor(images=image, text=prompt, return_tensors="pt")
        inputs = inputs.to(device)

        with torch.no_grad():

            outputs = model.generate(**inputs)
            pred_short = processor.decode(outputs[0], skip_special_tokens=True)

            
            
            if device=="cuda":
                torch.cuda.empty_cache()

        pred_short_clean = clean_text(pred_short)
        gt_clean = clean_text(str(gt))

        # Compute similarity only if both GT and prediction exist
        if pred_short_clean and gt_clean:
            emb_short = baai.encode(pred_short_clean, convert_to_tensor=True, show_progress_bar=False)
            emb_gt = baai.encode(gt_clean, convert_to_tensor=True, show_progress_bar=False)
            sim_short = min(1.0, round(util.cos_sim(emb_short, emb_gt).item(), 3))
        else:
            sim_short = ""
        
        if pred_short_clean and gt_clean:
            P, R, F1 = score([pred_short_clean], [gt_clean], lang="en", verbose=False)
            bert_score_f1 = round(F1.item(), 4)
            bert_score_precision = round(P.item(), 4)
            bert_score_recall = round(R.item(), 4)
        else:
            bert_score_f1 = ""
        
        # Exact match only if both exist
        exact_match = 1 if pred_short_clean and gt_clean and (pred_short_clean == gt_clean) else 0                    

        results.append({
            "index": i,
            "image_path": image_path,
            "question": question,
            "answer": gt,
            "blipvqa_prediction_short": pred_short_clean,
            "baai_similarity_short": sim_short,
            "bert_score_f1": bert_score_f1,
            "bert_score_precision":bert_score_precision,
            "bert_score_recall":bert_score_recall,
            "blipvqa_exact_match_short": exact_match
        })
        
        if i%50 == 0:
            print(f"{i} rows complete")

        if (i - start_row + 1) % 1000 == 0:
            pd.DataFrame(results).to_csv(output_csv, mode='a', index=False, header=not os.path.exists(output_csv))
            print(f"[SAVED] Up to row {i}")
            results = []

    except Exception as e:
        print(f"[ERROR] at row {i}: {e}")
        continue

# Final save
if results:
    pd.DataFrame(results).to_csv(output_csv, mode='a', index=False, header=not os.path.exists(output_csv))
    print("Download it dude !!!!!!!!!!")

5050 rows complete
5100 rows complete
5150 rows complete
5200 rows complete
5250 rows complete
5300 rows complete
5350 rows complete
